In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from collections import Counter as ctr
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df = pd.read_csv('/kaggle/input/symptom2disease/Symptom2Disease.csv')
df.head(10)

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."
5,5,Psoriasis,The skin on my palms and soles is thickened an...
6,6,Psoriasis,"The skin around my mouth, nose, and eyes is re..."
7,7,Psoriasis,My skin is very sensitive and reacts easily to...
8,8,Psoriasis,I have noticed a sudden peeling of skin at dif...
9,9,Psoriasis,The skin on my genitals is red and inflamed. I...


In [4]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [5]:
ctr(df['label'])

Counter({'Psoriasis': 50,
         'Varicose Veins': 50,
         'Typhoid': 50,
         'Chicken pox': 50,
         'Impetigo': 50,
         'Dengue': 50,
         'Fungal infection': 50,
         'Common Cold': 50,
         'Pneumonia': 50,
         'Dimorphic Hemorrhoids': 50,
         'Arthritis': 50,
         'Acne': 50,
         'Bronchial Asthma': 50,
         'Hypertension': 50,
         'Migraine': 50,
         'Cervical spondylosis': 50,
         'Jaundice': 50,
         'Malaria': 50,
         'urinary tract infection': 50,
         'allergy': 50,
         'gastroesophageal reflux disease': 50,
         'drug reaction': 50,
         'peptic ulcer disease': 50,
         'diabetes': 50})

In [6]:
df.sample(10)

,label,text
239,Impetigo,"A rash has developed around my nose and lips, ..."
90,Varicose Veins,I have a rash on my legs that is giving me a l...
697,Hypertension,I was in the middle of a meeting when I sudden...
262,Dengue,The chills and shivering I have been experienc...
169,Chicken pox,There are red spots all over my body that I ca...
121,Typhoid,I'm having severe stomach pain and constipatio...
1011,gastroesophageal reflux disease,I have indigestion and heartburn. I frequently...
843,Jaundice,"I've been experiencing intense itchiness, naus..."
703,Migraine,"I've been suffering visual disruptions, seeing..."
692,Hypertension,"I felt fine all day, but when I was driving ho..."


In [7]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

df['text'] = df['text'].apply(preprocess_text)

In [8]:
df.sample(5)

,label,text
1116,peptic ulcer disease,i have stomach ach and bend over or lie down j...
1090,drug reaction,i no longer want to have sex and it difficult ...
871,Malaria,along with my intens scratch chill nausea and ...
542,Arthritis,i been experienc stiff and weak in my neck mus...
309,Fungal infection,on my arm and leg i have a lot of red pimpl an...


In [9]:
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['text'])

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
base_models = [
    ('nb', MultinomialNB()),
    ('rf', RandomForestClassifier()),
    ('lr', LogisticRegression()),
    ('svm', SVC(kernel='linear', probability=True))
]

In [12]:
voting_classifier = VotingClassifier(estimators=base_models, voting='hard')

In [13]:
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('nb', MultinomialNB()),
                             ('rf', RandomForestClassifier()),
                             ('lr', LogisticRegression()),
                             ('svm', SVC(kernel='linear', probability=True))])

In [14]:
accuracy = voting_classifier.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9708333333333333


In [15]:
import joblib

joblib.dump(voting_classifier, 'voting_classifier_model_Disease_pred_97_percent_acc.pkl')

['voting_classifier_model_Disease_pred_97_percent_acc.pkl']

In [16]:
loaded_model = joblib.load('/kaggle/working/voting_classifier_model_Disease_pred_97_percent_acc.pkl')

In [17]:
# Sample text
sample_text = "I have been experiencing a skin rash on my arm for the past few weeks."
sample_text_processed = preprocess_text(sample_text)
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])
predicted_label = label_encoder.inverse_transform(voting_classifier.predict(sample_text_transformed))

print("Predicted Label:", predicted_label)

Predicted Label: ['Psoriasis']


In [18]:
text = 'i been realli weari and ill i been suffer from..'

sample_text = text
sample_text_processed = preprocess_text(sample_text)
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])
predicted_label = label_encoder.inverse_transform(voting_classifier.predict(sample_text_transformed))

print("Predicted Label:", predicted_label)

Predicted Label: ['Common Cold']


In [19]:
df.sample()

,label,text
8,Psoriasis,i have notic a sudden peel of skin at differ p...


In [20]:
# Compute accuracy
accuracy = accuracy_score(y_test, y_test)
print("Accuracy:", accuracy)

# Compute precision
precision = precision_score(y_test, y_test, average='macro')  # 'macro' computes precision for each label, and returns the average
print("Precision:", precision)

# Compute recall
recall = recall_score(y_test, y_test, average='macro')  # 'macro' computes recall for each label, and returns the average
print("Recall:", recall)

# Compute F1-score
f1 = f1_score(y_test, y_test, average='macro')  # 'macro' computes F1-score for each label, and returns the average
print("F1-score:", f1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_test)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1-score: 1.0
Confusion Matrix:
[[ 7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  

In [21]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer_disease_nlp.joblib')
joblib.dump(label_encoder, 'label_encoder_disease_nlp.joblib')

['label_encoder_disease_nlp.joblib']

In [22]:
import joblib
voting_classifier = joblib.load('/kaggle/working/voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('/kaggle/working/tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('/kaggle/working/label_encoder_disease_nlp.joblib')

In [23]:
import joblib

# Load the model and vectorizer
voting_classifier = joblib.load('/kaggle/working/voting_classifier_model_Disease_pred_97_percent_acc.pkl')
tfidf_vectorizer = joblib.load('/kaggle/working/tfidf_vectorizer_disease_nlp.joblib')
label_encoder = joblib.load('/kaggle/working/label_encoder_disease_nlp.joblib')

# Sample text
sample_text = "I have been experiencing a skin rash on my arm for the past few weeks."

# Preprocess the sample text
def preprocess_text(text):
    tokens = word_tokenize(text)
    snowball_stemmer = SnowballStemmer('english')
    tokens = [snowball_stemmer.stem(token.lower()) for token in tokens if token.isalpha()]
    return ' '.join(tokens)

sample_text_processed = preprocess_text(sample_text)

# Transform the preprocessed sample text using the loaded vectorizer
sample_text_transformed = tfidf_vectorizer.transform([sample_text_processed])

# Predict using the loaded model
predicted_label_encoded = voting_classifier.predict(sample_text_transformed)

# Decode the predicted label
predicted_label = label_encoder.inverse_transform(predicted_label_encoded)

print("Predicted Label:", predicted_label)

Predicted Label: ['Psoriasis']
